# Definição

Este notebook tem como principal função estruturar uma forma de organizar dados obtidos por meio de busca de artigos (revisão). Para automatizar este processo, o buscador Google Scholar foi escolhido. Como não estava disponível uma API para extração de dados, foi desenvolvido um Web Crawler para extrair os dados de artigos encontrados pelo buscador, organizando-os em uma planilha eletrônica para manipulação e consulta posterior.

A lista de queries é a principal variável de controle deste notebook, uma vez que define quais serão os termos pesquisados no buscador. Por definição, dado que o objetivo não é uma revisão sistemática, a requisição utiliza como padrão o algoritmo de ordenação por relevância do Google e acessa até os 70 primeiros resultados por query.

OBS: O Google pode bloquear requisições feitas por scripts, para isso uma VPN soluciona. Foi utilizada a Proton VPN quando necessário.

In [2]:
import os
import requests
import pandas as pd
from typing import List
from zipfile import ZipFile
from bs4 import BeautifulSoup
from datetime import date, datetime

BASE_PATH = os.path.dirname(os.getcwd())

/tmp/ipykernel_35529/3914482469.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Caminhos e diretórios

Este bloco tem como função preparar os diretórios e salvar caminhos de arquivos em variáveis utilizadas posteriormente. Para se manter um certo rastreio, cada dia em que este notebook for executado representará uma pasta em dados.

As páginas HTML lidas são salvas em um diretório para que sejam processadas posteriormente e componham um histórico de execução dos processos. Um arquivo excel é gerado ao final com o resumo do processamento, bem como informações da execução.

In [2]:
# Gerencia os diretórios e caminho de arquivo excel
today = date.today().isoformat()
today_path = f"{BASE_PATH}/data/google_search/{today}"
pages_path = f"{BASE_PATH}/data/google_search/{today}/pages"
excel_path = f"{BASE_PATH}/data/google_search/{today}/search_results.xlsx"

if not os.path.isdir(today_path):
    os.mkdir(today_path)

if not os.path.isdir(pages_path):
    os.mkdir(pages_path)

## Parâmetros de execução

Este bloco é onde existe a customização da execução. O principal parâmetro é a lista com queries que serão buscadas. O valor de resultados por página deve ser mantido em 10 (padrão utilizado pelo Google Scholar na sua página gratuita). A quantidade máxima de resultados deve ser um múltiplo de 10 e pode ser alterada de acordo com a necessidade.

In [3]:
# Parâmetro para definir as queries que serão utilizadas na pesquisa
queries = [
    'agriculture+and+"linear+programming"',
    '"crops+pattern"+and+"linear+programming"',
    '"crop+rotation"+and+"linear+programming"',
    '"land+allocation"+and+"linear+programming"',
    '"land+allocation"+and+"linear+programming"+and+"crop-livestock"'
]

# Parâmetro para controle do limite de textos que serão extraídos.
# O Google Scholar exibe 10 resultados por página, portanto o limite
# de finalização representa, também, até que página será acessada.
per_page = 10
finish_at = 60

## Download de páginas

Para se ter um melhor controle do processamento, a primeira etapa consiste em fazer o download das páginas com resultados e armazená-las internamente. A variável de diretório definida na seção "Caminhos e diretórios" é utilizada aqui. Cada arquivo é salvo com o padrão "query_{numero_query}\_page\_{numero_pagina}.html". Utiliza um dicionário de log para armazenar dados de execução, utilizado na próxima etapa.

In [4]:
save_files = dict()

for i in range(0, len(queries)):
    query = queries[i]
    start_at = datetime.now()
    save_files[i] = {"duration": None, "start": None, "end": None, "files": []}
    
    for page in range(0, finish_at + per_page, per_page):
        # Exibe log de processamento
        page_number = 1 + int(page / per_page)
        print(f"Download Query {i + 1}, página {page_number}", end="\r")

        # Estrutura a url do google scholar para download de arquivos
        # Utiliza como ordenador a função de relevância do algoritmo do Google
        # Filtra apenas artigos publicados a partir de 2020
        # Não inclui resultados que são apenas citações, todo retorno é de material original
        url = f"https://scholar.google.com/scholar?lr=&q={query}&as_sdt=0,5&as_ylo=2020&as_vis=1&start={page}"
        req = requests.get(url)

        # Instancia web crawler e acessa listagem de resultados da busca
        soup = BeautifulSoup(req.content, "html.parser")
        search_results = soup.findAll("div", class_="gs_r gs_or gs_scl")

        # Não existem resultados na página, encerra o laço para paginação
        if len(search_results) == 0:
            break

        # Salva o arquivo para leitura posterior
        file_name = f"{pages_path}/query_{i + 1}_page_{page_number}.html"
        with open(file_name, "w+") as file:
            file.write(str(soup.prettify()))

        # Registra arquivo no log
        save_files[i]["files"].append(file_name)

    # Registra duração do processo
    end_at = datetime.now()
    save_files[i]["end"] = end_at.isoformat()
    save_files[i]["start"] = start_at.isoformat()
    save_files[i]["duration"] = (end_at - start_at).seconds

## Processamento

A partir do log de execução anterior, acessa páginas e instancia o web crawler que estrutura resultados em tabelas para serem salvos em excel de controle. Utiliza uma lista para armazenar dados tabulados dos resultados de cada query, assim como para armazenar dados sumarizados da execução da busca.

In [8]:
# Variáveis de armazenamento
summary: List[dict] = []
results: List[List[dict]] = []

for i in range(0, len(queries)):
    query_data: List[dict] = []
    
    for file_name in save_files[i]["files"]:
        # Acessa arquivo salvo e instancia o web crawler
        with open(file_name, "r") as file:
            soup = BeautifulSoup(file, "html.parser")
            search_results = soup.findAll("div", class_="gs_r gs_or gs_scl")

        # Percorre a lista de resultados, armazenando título do trabalho,
        # link de acesso e autores em lista própria
        for div in search_results:
            title = div.find("h3").find("a")
            author = div.find("div", class_="gs_a").text

            query_data.append({
                "Título": title.text.replace("\n", "").replace("  ", "").strip(),
                "Autor": author.replace("\n", "").replace("  ", "").strip(),
                "Link": title["href"]
            })

    # Adiciona leituras da query na lista de processados
    results.append(query_data)

    # Atualiza log de sumarização de busca
    summary.append({
        "Query": i + 1,
        "Texto": queries[i],
        "Quantidade de artigos": len(query_data),
        "Início em": save_files[i]["start"],
        "Término em": save_files[i]["end"],
        "Duração (s)": save_files[i]["duration"]
    })

## Salva arquivo resumo

A última etapa deste notebook é compilar todos os dados em um arquivo excel de resumo, contendo as queries utilizadas, tempo para download dos resultados, quantidade de textos encontrados e a união de todos os textos.

In [9]:
with pd.ExcelWriter(excel_path, mode="w") as writer:
    # Salva sumário da busca, com textos de queries
    # Inicia variável para união de resultados
    pd.DataFrame(summary).to_excel(writer, sheet_name="summary", index=False)
    union: List[dict] = []

    # Define uma forma de busca na lista de união
    def search(title: str) -> dict | None:
        element = None
        for i in range(0, len(union)):
            if union[i]["Título"] == title:
                element = union[i]
                break
        return element

    for i in range(0, len(results)):
        # Salva query isolada no arquivo excel
        query_data = results[i]
        pd.DataFrame(query_data).to_excel(writer, sheet_name=f"query_{i + 1}", index=False)

        # Faz o processamento na união
        for record in query_data:
            element = search(record["Título"])
            if element is None:
                record["Query"] = i + 1
                union.append(record)
            else:
                element["Query"] = f'{element["Query"]},{i + 1}'

    # Salva união no arquivo excel
    pd.DataFrame(union).to_excel(writer, sheet_name="union", index=False)

## Comprime páginas

Para evitar a quantidade de arquivos HMTL neste diretório, o próximo bloco adiciona todos os arquivos baixados em um .zip e remove os arquivos originais.

In [13]:
# Coloca páginas HTML em arquivo zip
files = os.listdir(pages_path)
with ZipFile(f"{pages_path}/pages.zip", "w") as zip_file:
    for file in files:
        zip_file.write(f"{pages_path}/{file}")

# Remove páginas HTML
for file in files:
    os.remove(f"{pages_path}/{file}")